In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizer

from vllm import LLM

INFO 09-26 20:36:12 [__init__.py:244] Automatically detected platform rocm.


#### Hugging Face API access

In [2]:
from huggingface_hub import notebook_login, HfApi

notebook_login()

In [3]:
try:
    api = HfApi()
    user_info = api.whoami()
    print(f"Token validated successfully! Logged in as: {user_info['name']}")
except Exception as e:
    print(f"Token validation failed. Error: {e}")

Token validated successfully! Logged in as: Chamath


### Initilize

In [4]:
def init_tokenizer_and_llm(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    transformers_model = AutoModelForCausalLM.from_pretrained(model_name)
    embedding_layer = transformers_model.get_input_embeddings()
    llm = LLM(model=model_name, enable_prompt_embeds=True)
    return tokenizer, embedding_layer, llm

### Prompt Embeddings

In [13]:
def get_prompt_embeds(
    chat: list[dict[str, str]],
    tokenizer: PreTrainedTokenizer,
    embedding_layer: torch.nn.Module,
):
    token_ids = tokenizer.apply_chat_template(
        chat, add_generation_prompt=True, return_tensors="pt"
    )
    prompt_embeds = embedding_layer(token_ids).squeeze(0)
    return prompt_embeds

### Inference

In [6]:
def single_prompt_inference(
    llm: LLM, tokenizer: PreTrainedTokenizer, embedding_layer: torch.nn.Module
):
    chat = [{"role": "user", "content": "Please tell me about the capital of France."}]
    prompt_embeds = get_prompt_embeds(chat, tokenizer, embedding_layer)

    outputs = llm.generate(
        {
            "prompt_embeds": prompt_embeds,
        }
    )

    print("\n[Single Inference Output]")
    print("-" * 30)
    for o in outputs:
        print(o.outputs[0].text)
    print("-" * 30)

In [7]:
def batch_prompt_inference(
    llm: LLM, tokenizer: PreTrainedTokenizer, embedding_layer: torch.nn.Module
):
    chats = [
        [{"role": "user", "content": "Please tell me about the capital of France."}],
        [{"role": "user", "content": "When is the day longest during the year?"}],
        [{"role": "user", "content": "Where is bigger, the moon or the sun?"}],
    ]

    prompt_embeds_list = [
        get_prompt_embeds(chat, tokenizer, embedding_layer) for chat in chats
    ]

    outputs = llm.generate([{"prompt_embeds": embeds} for embeds in prompt_embeds_list])

    print("\n[Batch Inference Outputs]")
    print("-" * 30)
    for i, o in enumerate(outputs):
        print(f"Q{i + 1}: {chats[i][0]['content']}")
        print(f"A{i + 1}: {o.outputs[0].text}\n")
    print("-" * 30)

In [10]:
model_name = "Qwen/Qwen2-7B-Instruct"

tokenizer, embedding_layer, llm = init_tokenizer_and_llm(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

INFO 09-26 20:39:27 [config.py:853] This model supports multiple tasks: {'generate', 'score', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 09-26 20:39:27 [config.py:1467] Using max model len 32768
WARNING 09-26 20:39:27 [arg_utils.py:1719] --enable-prompt-embeds is not supported by the V1 Engine. Falling back to V0. 
INFO 09-26 20:39:35 [config.py:4566] full_cuda_graph is not supported with cascade attention. Disabling cascade attention.
INFO 09-26 20:39:35 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2.dev364+gb432b7a28) with config: model='Qwen/Qwen2-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-26 20:39:42 [default_loader.py:272] Loading weights took 5.50 seconds
INFO 09-26 20:39:43 [model_runner.py:1203] Model loading took 14.5020 GiB and 5.967543 seconds
INFO 09-26 20:40:11 [worker.py:294] Memory profiling takes 27.96 seconds
INFO 09-26 20:40:11 [worker.py:294] the current vLLM instance can use total_gpu_memory (191.69GiB) x gpu_memory_utilization (0.90) = 172.52GiB
INFO 09-26 20:40:11 [worker.py:294] model weights take 14.50GiB; non_torch_memory takes 0.71GiB; PyTorch activation peak memory takes 4.76GiB; the rest of the memory reserved for KV Cache is 152.55GiB.
INFO 09-26 20:40:11 [executor_base.py:113] # rocm blocks: 178521, # CPU blocks: 4681
INFO 09-26 20:40:11 [executor_base.py:118] Maximum concurrency for 32768 tokens per request: 87.17x
INFO 09-26 20:40:12 [model_runner.py:1513] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-ea

Capturing CUDA graph shapes:   0%|          | 0/70 [00:00<?, ?it/s]

INFO 09-26 20:40:31 [model_runner.py:1671] Graph capturing finished in 19 secs, took 0.39 GiB
INFO 09-26 20:40:31 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 48.63 seconds


In [11]:
single_prompt_inference(llm, tokenizer, embedding_layer)

WARNING 09-26 20:41:52 [config.py:1394] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0% 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


[Single Inference Output]
------------------------------
The capital of France is Paris. It is located in the northern central part of
------------------------------


In [12]:
batch_prompt_inference(llm, tokenizer, embedding_layer)

Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0% 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


[Batch Inference Outputs]
------------------------------
Q1: Please tell me about the capital of France.
A1: The capital of France is Paris. It is located in the northern central part of

Q2: When is the day longest during the year?
A2: The day is longest during the year on the summer solstice. This typically occurs

Q3: Where is bigger, the moon or the sun?
A3: The Sun is much bigger than the Moon. Despite the Moon being the largest satellite

------------------------------
